# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [6]:
# 1) Contar o número de registros e verificar duplicatas
total_registros = sinasc.shape[0]
total_registros_unicos = sinasc.drop_duplicates().shape[0]
print(f"Total de registros: {total_registros}")
print(f"Total de registros únicos: {total_registros_unicos}")

# Verificar se há linhas duplicadas
if total_registros == total_registros_unicos:
    print("Não há linhas duplicadas.")
else:
    print("Há linhas duplicadas na base de dados.")

Total de registros: 27028
Total de registros únicos: 27028
Não há linhas duplicadas.


In [10]:
# 2) Contar o número de valores missing por variável
missing_values = sinasc.isnull().sum()
print("\nValores ausentes por variável:\n", missing_values)


Valores ausentes por variável:
 ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [12]:
# Selecionar apenas as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interesse]

# Contar valores missing por variável na base selecionada
missing_values_selecionado = sinasc_selecionado.isnull().sum()
print("Valores ausentes por variável (colunas selecionadas):\n", missing_values_selecionado)

Valores ausentes por variável (colunas selecionadas):
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [14]:
# Remover registros com Apgar5 não preenchido
sinasc_selecionado = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção
total_registros_filtrados = sinasc_selecionado.shape[0]
print(f"Total de registros após remoção de Apgar5 não preenchido: {total_registros_filtrados}")

# Contar valores missing por variável na base filtrada
missing_values_filtrado = sinasc_selecionado.isnull().sum()
print("\nValores ausentes por variável (após remoção de Apgar5 ausente):\n", missing_values_filtrado)


Total de registros após remoção de Apgar5 não preenchido: 26925

Valores ausentes por variável (após remoção de Apgar5 ausente):
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [18]:
# Substituir valores ausentes por 9 nas colunas ESTCIVMAE e CONSULTAS, sem usar inplace
sinasc_selecionado['ESTCIVMAE'] = sinasc_selecionado['ESTCIVMAE'].fillna(9)
sinasc_selecionado['CONSULTAS'] = sinasc_selecionado['CONSULTAS'].fillna(9)

# Contar novamente os valores missing por variável para confirmar as mudanças
missing_values_final = sinasc_selecionado.isnull().sum()
print("\nValores ausentes por variável (após substituição por 9):\n", missing_values_final)


Valores ausentes por variável (após substituição por 9):
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [20]:
# Substituir valores ausentes por 0 na coluna QTDFILVIVO
sinasc_selecionado['QTDFILVIVO'] = sinasc_selecionado['QTDFILVIVO'].fillna(0)

# Contar novamente os valores missing por variável para confirmar as mudanças
missing_values_final = sinasc_selecionado.isnull().sum()
print("\nValores ausentes por variável (após substituição de QTDFILVIVO por 0):\n", missing_values_final)


Valores ausentes por variável (após substituição de QTDFILVIVO por 0):
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [24]:
# Substituir valores ausentes conforme as decisões acima

# LOCNASC: preencher com o valor mais comum
sinasc_selecionado['LOCNASC'] = sinasc_selecionado['LOCNASC'].fillna(sinasc_selecionado['LOCNASC'].mode()[0])

# IDADEMAE: preencher com a mediana
sinasc_selecionado['IDADEMAE'] = sinasc_selecionado['IDADEMAE'].fillna(sinasc_selecionado['IDADEMAE'].median())

# ESCMAE: preencher com 9 para indicar ignorado
sinasc_selecionado['ESCMAE'] = sinasc_selecionado['ESCMAE'].fillna(9)

# GESTACAO: preencher com o valor mais comum
sinasc_selecionado['GESTACAO'] = sinasc_selecionado['GESTACAO'].fillna(sinasc_selecionado['GESTACAO'].mode()[0])

# GRAVIDEZ: preencher com o valor mais comum
sinasc_selecionado['GRAVIDEZ'] = sinasc_selecionado['GRAVIDEZ'].fillna(sinasc_selecionado['GRAVIDEZ'].mode()[0])

# Contar valores missing por variável para confirmar as mudanças
missing_values_final = sinasc_selecionado.isnull().sum()
print("\nValores ausentes por variável (após preenchimento final):\n", missing_values_final)


Valores ausentes por variável (após preenchimento final):
 LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [26]:
# Criar uma nova coluna 'ASFIXIA' com a categorização de Apgar5
sinasc_selecionado['ASFIXIA'] = pd.cut(
    sinasc_selecionado['APGAR5'],
    bins=[-1, 3, 5, 7, 10],  # Intervalos de acordo com a classificação
    labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']
)

# Calcular a frequência das categorias de asfixia
frequencias_asfixia = sinasc_selecionado['ASFIXIA'].value_counts()
print("Frequências da categorização de Apgar5:\n", frequencias_asfixia)

Frequências da categorização de Apgar5:
 ASFIXIA
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [28]:
# Renomear as colunas para snake_case
sinasc_selecionado.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'ASFIXIA': 'asfixia'
}, inplace=True)

# Verificar as mudanças de nome
print("Nomes das colunas após a renomeação:\n", sinasc_selecionado.columns)

Nomes das colunas após a renomeação:
 Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'asfixia'],
      dtype='object')
